# Anomaly Detection Models

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
plt.style.use('ggplot')
#pd.set_option('max_columns', 200)
from sklearn.model_selection import train_test_split

In [2]:
path = '../data/data_preparation_output.csv'  # ASY local path
# path = "../data/skylab_instagram_datathon_dataset.csv" # gen
df = pd.read_csv(path) #sep='delimiter', header=None) # sep=";", 

In [3]:
df.head()

,period_end_date,compset_group,business_entity_doing_business_as_name,legal_entity_name,domicile_country_name,ultimate_parent_legal_entity_name,primary_exchange_name,followers,pictures,videos,...,timediff_1,diff_1_followers,lag_1_pictures,diff_1_pictures,lag_1_videos,diff_1_videos,lag_1_comments,diff_1_comments,Country Name,Gene Index
0,2017-05-06,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN
1,2017-05-13,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,6.0,3.0,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-05-20,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,6.0,3.0,...,1.0,NaN,6.0,0.0,3.0,0.0,57.0,0.0,NaN,NaN
3,2017-05-27,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,6.0,3.0,...,1.0,NaN,6.0,0.0,3.0,0.0,57.0,0.0,NaN,NaN
4,2017-06-03,apparel retail,24s,lvmh moet hennessy louis vuitton se,france,lvmh moet hennessy louis vuitton se,euronext paris,NaN,24.0,3.0,...,1.0,NaN,6.0,18.0,3.0,0.0,57.0,52.0,NaN,NaN


## Approaches

1. https://scikit-learn.org/stable/modules/outlier_detection.html#novelty-detection

In [5]:
df.columns

Index(['period_end_date', 'compset_group',
       'business_entity_doing_business_as_name', 'legal_entity_name',
       'domicile_country_name', 'ultimate_parent_legal_entity_name',
       'primary_exchange_name', 'followers', 'pictures', 'videos', 'comments',
       'likes', 'compset', 'Year', 'Month', 'Day', 'Weekday',
       'ultimate_parent_vs_legal_entity', 'ultimate_parent_vs_business_entity',
       'legal_entity_vs_business_entity', 'same_ownership',
       'total_involvement', 'total_company_activity', 'conversion_rate_total',
       'return_on_activity', 'ratio_of_videos', 'ratio_of_pictures',
       'likes_per_picture', 'likes_per_video', 'comments_per_picture',
       'comments_per_video',
       'business_entity_doing_business_as_name_cnt_industry',
       'business_entity_doing_business_as_name_cnt_industry_country',
       'followers_sum_industry', 'videos_sum_industry',
       'pictures_sum_industry', 'likes_sum_industry', 'comments_sum_industry',
       'followers_sum_

In [7]:
cols_to_ignore = ['compset', 'period_end_date', 'lag_1_date', 'lag_1_company']
cols_of_interest = ['followers', 'pictures', 'videos', 'comments',
       'likes', 'Year', 'Month', 
       'ultimate_parent_vs_legal_entity', 'ultimate_parent_vs_business_entity',
       'legal_entity_vs_business_entity', 'same_ownership',
       'total_involvement', 'total_company_activity', 'conversion_rate_total',
       'return_on_activity', 'ratio_of_videos', 'ratio_of_pictures',
       'likes_per_picture', 'likes_per_video', 'comments_per_picture',
       'comments_per_video',
       'business_entity_doing_business_as_name_cnt_industry',
       'business_entity_doing_business_as_name_cnt_industry_country',
       'followers_sum_industry', 'videos_sum_industry',
       'pictures_sum_industry', 'likes_sum_industry', 'comments_sum_industry',
       'followers_sum_industry_country', 'videos_sum_industry_country',
       'pictures_sum_industry_country', 'likes_sum_industry_country',
       'comments_sum_industry_country', 'fraction_followers_sum_industry',
       'fraction_followers_sum_industry_country',
       'fraction_videos_sum_industry', 'fraction_videos_sum_industry_country',
       'fraction_pictures_sum_industry',
       'fraction_pictures_sum_industry_country', 'fraction_likes_sum_industry',
       'fraction_likes_sum_industry_country', 'fraction_comments_sum_industry',
       'fraction_comments_sum_industry_country', 'lag_1_followers',
        'timediff_1', 'diff_1_followers',
       'lag_1_pictures', 'diff_1_pictures', 'lag_1_videos', 'diff_1_videos',
       'lag_1_comments', 'diff_1_comments'] 

In [8]:
df_analysis = df[cols_of_interest]
no_nans = df_analysis[~df_analysis.isnull().any(axis=1)]
print(f"{len(df_analysis)} vs {len(no_nans)}")

298040 vs 133998


In [14]:
import random
sample_size = 10000
rand_idx = random.sample(list(no_nans.index), sample_size)
df_short = no_nans.loc[rand_idx]

### SVM

https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html#sphx-glr-auto-examples-svm-plot-oneclass-py

In [ ]:
from sklearn import svm

clf_svm = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)

In [15]:
clf_svm.fit(df_short)

OneClassSVM(gamma=0.1, nu=0.1)

### LOF

In [16]:
from sklearn.neighbors import LocalOutlierFactor

1. DBScan
2. Isolation Forest
3. LOF

In [17]:
clf_lof = LocalOutlierFactor(n_neighbors=20, novelty=True, contamination=0.1)
clf_lof.fit(df_short)

LocalOutlierFactor(contamination=0.1, novelty=True)

### DBSCan

In [24]:
from sklearn import preprocessing

d = preprocessing.normalize(df_short)
scaled_df_short = pd.DataFrame(d, columns=df_short.columns)
#scaled_df_short.head()

In [26]:
from sklearn import metrics
from sklearn.cluster import DBSCAN

db = DBSCAN(eps=0.3, min_samples=10).fit(scaled_df_short)
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print("Estimated number of clusters: %d" % n_clusters_)
print("Estimated number of noise points: %d" % n_noise_)

Estimated number of clusters: 1
Estimated number of noise points: 7


In [30]:
#eps_set = [0.01, 0.1, 0.3, 0.5, 1]
eps_set = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.1]

for eps in eps_set :
    db = DBSCAN(eps=eps, min_samples=10).fit(scaled_df_short)
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    print(eps)
    print("Estimated number of clusters: %d" % n_clusters_)
    print("Estimated number of noise points: %d" % n_noise_)

0.01
Estimated number of clusters: 61
Estimated number of noise points: 4365
0.02
Estimated number of clusters: 46
Estimated number of noise points: 1961
0.03
Estimated number of clusters: 32
Estimated number of noise points: 1113
0.04
Estimated number of clusters: 18
Estimated number of noise points: 745
0.05
Estimated number of clusters: 13
Estimated number of noise points: 526
0.06
Estimated number of clusters: 13
Estimated number of noise points: 370
0.1
Estimated number of clusters: 6
Estimated number of noise points: 126


In [29]:
min_samples_set=[5, 10, 15]

for min_samples in min_samples_set:
    for 
    db = DBSCAN(eps=0.1, min_samples=min_samples).fit(scaled_df_short)
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)
    print(min_samples)
    print("Estimated number of clusters: %d" % n_clusters_)
    print("Estimated number of noise points: %d" % n_noise_)

5
Estimated number of clusters: 10
Estimated number of noise points: 54
10
Estimated number of clusters: 6
Estimated number of noise points: 126
15
Estimated number of clusters: 3
Estimated number of noise points: 185


### Isolated Forest

